# Pytorch

pytorch is a python library for deep learning. It is built on top of the CPU and GPU.

## Installation

```bash
pip install torch
```

## Import

```python
import torch
```

With pytorch you can create a model and train it. You can also create datasets object and iterate over them

In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
import pandas as pd
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm

In [41]:


def transform_flatten_normalize(image: torch.Tensor) -> torch.Tensor:
    """flattens the image tensor
    """
    image = image.flatten() # flatten the image for a multi-dimensional input to a vecture of shape (1, 28*28)
    image = image.float() # convert the flattened image to a float tensor to be able to normalize it
    
    # normalize the image to be between 0 and 1 we devide by 255 because the image is in the range of 0 to 255
    image = image / 255.0
    
    return image

def read_image_torch(image_path: str) -> torch.Tensor:
    """reads the image from the path and returns a tensor
    """
    image = Image.open(image_path)
    array_image = np.array(image)
    return torch.as_tensor(array_image)

# def tranform_label_to_one_hot(label:int):
#     """transforms the label to a one hot vector of shape (1, 10) because we have 10 classes
#     """
#     return torch.tensor(label, dtype=torch.f)

def read_image_to_labels_file(annotations_file: str, train: bool) -> pd.DataFrame:
    img_labels = pd.read_csv(annotations_file)
    img_labels = img_labels[img_labels['train'] == train]
    return img_labels

class MnistDataset(Dataset):
    """A Pytorch Dataset class for the MNIST dataset
    """
    def __init__(self, annotations_file, images_folder, train, transform=None, target_transform=None):
        self.img_labels = read_image_to_labels_file(annotations_file, train)
        self.images_folder = images_folder
        self.transform = transform
        self.target_transform = target_transform

    # we always need to define the __len__ method for the dataset
    def __len__(self):
        return len(self.img_labels)

    # we always need to define the __getitem__ method for the dataset
    def __getitem__(self, idx):
        img_path = os.path.join(self.images_folder, self.img_labels.iloc[idx, 0])
        image = read_image_torch(img_path)
        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

In [42]:
mnist_no_transform_dataset = MnistDataset(
    annotations_file="../data/mnist/mnist.csv",
    images_folder="../data/mnist",
    train=True,
    transform=transform_flatten_normalize,
    target_transform=None,
)

# Data loaders with pytorch

In [43]:
train_dataset = MnistDataset(
    annotations_file="../data/mnist/mnist.csv", 
    images_folder="../data/mnist/", 
    train=True, 
    transform=transform_flatten_normalize, 
    target_transform=None,
)
test_dataset = MnistDataset(
    annotations_file="../data/mnist/mnist.csv", 
    images_folder="../data/mnist/", 
    train=False, 
    transform=transform_flatten_normalize, 
    target_transform=None,
)
train_dataloader = DataLoader(train_dataset, batch_size=100, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=100, shuffle=True)

In [44]:
x, y = next(iter(train_dataloader))

In [ ]:
x.shape

In [ ]:
y

In [74]:
# create fully connected layer pytorch
from torch import nn
import torch.nn.functional as F

class SimpleNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(784, 10)
        self.fc2 = nn.Linear(10, 10)

    def forward(self, x):
        x = self.fc1(x)
        x = F.sigmoid(x)
        x = self.fc2(x)
        return x
    
class VerySimpleNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(784, 10)

    def forward(self, x):
        x = self.fc1(x)
        return x
    
# Evaluate the performance of the neural network
def evaluate(net, test_dataloader):
    ground_truths = []
    predictions = []
    net = net.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        eval_loss = 0
        for x, y in tqdm(test_dataloader):
            x = x
            y_pred = net(x)
            predicted = y_pred.argmax(dim=1)
            total += y.size(0)
            correct += (predicted == y).sum().item()
            eval_loss += F.cross_entropy(y_pred, y, reduction='sum').item()
            ground_truths.extend(y.tolist())
            predictions.extend(predicted.tolist())
        accuracy = correct / total
        eval_loss = eval_loss / total
        print(f'Eval Loss: {eval_loss}')
        print(f'Accuracy: {accuracy:.2f}')
        
    net = net.train()
    return ground_truths, predictions, accuracy, eval_loss

    

In [ ]:
very_simple_net = VerySimpleNet()
# train very_simple_net

learning_rate = 0.001 # to be tuned
optimizer = torch.optim.Adam(very_simple_net.parameters(), lr=learning_rate)

loss_fn = nn.CrossEntropyLoss()

summary_writer = SummaryWriter(log_dir='./runs/pytorch_fully_connected_mnist')

num_epochs = 10
for epoch in range(num_epochs):
    accumulated_loss = 0
    for x, y in tqdm(train_dataloader):
        # for each iteration, I get a batch of images of size (batch size 100) and a batch of labels
        
        y_pred = very_simple_net(x)

        # Compute and print loss
        loss = loss_fn(y_pred, y)
        
        accumulated_loss += loss.item()

        # Zero gradients, perform a backward pass, and update the weights.
        optimizer.zero_grad()
        
        loss.backward()
        optimizer.step()
    print(f'------- Epoch {epoch + 1}: Train Loss {accumulated_loss/len(train_dataloader)}')
    _, _, eval_acc, eval_loss = evaluate(very_simple_net, test_dataloader)
    summary_writer.add_scalars('Loss', {'train': accumulated_loss/len(train_dataloader), 'eval': eval_loss}, epoch)
    summary_writer.add_scalars('Accuracy', {'eval': eval_acc}, epoch)

summary_writer.close()

In [ ]:
simple_net = SimpleNet()
# train simple_net

learning_rate = 0.001
optimizer = torch.optim.Adam(simple_net.parameters(), lr=learning_rate)

loss_fn = nn.CrossEntropyLoss()

num_epochs = 40
simple_net = simple_net.to('cuda')
for epoch in range(num_epochs):
    accumulated_loss = 0
    for x, y in tqdm(train_dataloader):
        x = x.to('cuda')
        y = y.to('cuda')
        # Forward pass: Compute predicted y by passing x to the model
        y_pred = simple_net(x)

        # Compute and print loss
        loss = loss_fn(y_pred, y)
        accumulated_loss += loss.item()

        # Zero gradients, perform a backward pass, and update the weights.
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'------Epoch {epoch + 1}: Train Loss {accumulated_loss/len(train_dataloader)}')
    evaluate(simple_net, test_dataloader)


In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay, classification_report
ground_truths, predictions = evaluate(very_simple_net, test_dataloader)
ConfusionMatrixDisplay.from_predictions(ground_truths, predictions)
plt.show()
print(classification_report(ground_truths, predictions))

In [ ]:
ground_truths, predictions = evaluate(simple_net, test_dataloader)
ConfusionMatrixDisplay.from_predictions(ground_truths, predictions)
plt.show()
print(classification_report(ground_truths, predictions))